In [73]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import pytz

In [101]:
class TimePattern:
    def __init__(self,pattern_path=None,tz='America/New_York'):
        self._set_timeZone(tz)
        self._load_mapping(pattern_path)
        
    
    def process(self, sentence):
        current = dt.datetime.now()
        current = self.tz.localize(current)
        fixymd = self.evl_ymd(sentence)
        selfdefine = re.findall(self.re_ext, sentence)
        result = []
        for each in fixymd:
            future = self.evl(each['expression'])
            gap = (future - current).total_seconds()
            result.append({'pattern':each['pattern'], 'time':future, 'gapS':gap, 'gapH':gap/3600})
        for each in selfdefine:
            future = self.evl(self.dict_ext[each])
            gap = (future - current).total_seconds()
            result.append({'pattern':each, 'time':future, 'gapS':gap, 'gapH':gap/3600})
        return result
    
    def evl(self, expression):
        current = dt.datetime.now()
        current = self.tz.localize(current)
        exp_week = re.findall(r'-.+W-.+w',expression)
        exp_ymd = re.findall(r'.+y-.+m.+d',expression)

        history = self._pros_second(expression, current)
        history = self._pros_minute(expression, current, history)
        history,shift = self._pros_hour(expression, current, history)

        if exp_week:
            history = self._pros_weekDay(expression,current, history)
            history = self._pros_week(expression,current, history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_W(history)  
        elif exp_ymd:
            history = self._pros_day(expression, current,history)
            history = self._pros_month(expression, current,history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_D(history)
        if future.tzinfo is None:
            future = self.tz.localize(future)
        if not shift:
            future = future - self.delta
        return future
    
    def _load_mapping(self, pattern_path):
        df = pd.read_csv('mapping.csv')
        # create length
        df['length'] = df.key_word.apply(lambda x: len(x))
        df = df.sort_values(['length','key_word'], ascending=False)
        df_series = pd.Series(index=df.key_word.values, data=df.expression.values)
        df_dict = df_series.to_dict()
        self.serires = df_series
        self.re_ext = r'|'.join(self.serires.index.values)
        self.dict_ext = df_dict
        
    def _set_timeZone(self,tz):
        utc = pytz.utc
        self.tz = pytz.timezone(tz)
        now = dt.datetime.now()
        utc_now = utc.localize(now)
        tz_now = self.tz.localize(now)
        delta =  utc_now - tz_now
        hours = round(delta.total_seconds() / 3600)
        self.delta = dt.timedelta(hours=hours)
        
    def _pros_second(self, expression, current, history={'microsecond':0}):
        history = history.copy()
        S = current.second
        reexp = r'M:.+S'
        extract = re.findall(reexp,expression)[0]
        # M:+1S
        if extract[2:-1] == '?':
            history.update({'second':S})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(seconds=gap) 
            create = create.replace(**history)
            return create 
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create =  current - dt.timedelta(seconds=gap)
            create = create.replace(**history)
            return create
        else:
            second = int(extract[2:-1])
            history.update({'second':second})
            return history
        
    def _pros_minute(self, expression, current, history = {}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        M = current.minute
        reexp = r'H:.+M'
        extract = re.findall(reexp,expression)[0]
        # H:?M
        if extract[2:-1] == '?':
            history.update({'minute':M})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        else:
            minute = int(extract[2:-1])
            history.update({'minute':minute})
            return history
        
    def _pros_hour(self, expression, current, history = {}):
        shift = True
        if isinstance(history,dt.datetime):
            return history, shift
        history = history.copy()

        H = current.hour
        reexp = r'[dw]-.+H'
        extract = re.findall(reexp,expression)[0]
    #     d-?H
        if extract[2:-1] == '?':
            history.update({'hour':H})
            return history,shift
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        else:
            shift = False
            hour = int(extract[2:-1])
            history.update({'hour':hour})
            return history,shift
        
    def _pros_day(self, expression, current, history={}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        d = current.day
        reexp = r'm-.+d'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'day':d})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        else:
            day = int(extract[2:-1])
            history.update({'day':day})
            return history
        
    def _pros_month(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = 0
        m = current.month
        reexp = r'y-.+m'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'month':m})
            return history
        elif extract[2] == '+':
            cur = int(extract[3:-1]) + m
            if cur > 12:
                adjust_year = int(cur / 12)
                cur = cur % 12
                if cur == 0:
                    cur = 12
                    adjust_year -= 1
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        elif extract[2] == '-':
            cur = m - int(extract[3:-1])
            if cur < 1:
                adjust_year = int(cur / 12) - 1
                cur = cur % 12
                if cur == 0:
                    cur = 12
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        else:
            history.update({'month':int(extract[2:-1])})
            return history
        
    def _pros_year(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = history.get('year')
        if adjust_year is None:
            adjust_year = 0
        y = current.year
        reexp = r'.+y-'
        extract = re.findall(reexp,expression)[0]
        if extract[0:-2] == '?':
            history.update({'year':y+adjust_year})
            return history
        elif extract[0] == '+':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year+gap})
            return history
        elif extract[0] == '-':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year-gap})
            return history
        else:
            history.update({'year':int(extract[:-2])+adjust_year})
            return history
        
    def _pros_weekDay(self, expression, current, history):
        history = history.copy()
        w = current.isocalendar()[2] 
        reexp = r'W-.+w'
        extract = re.findall(reexp,expression)[0]

        # W-+1w
        if extract[2:-1] == '?':
            history.update({'weekday':str(w)})
            return history
        elif extract[2] == '+':
            rep = str(w + int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        elif extract[2] == '-':
            rep = str(w - int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'weekday':rep})
            return history
        
    def _pros_week(self, expression, current, history):
        history = history.copy()
        year_adjust = 0
        W = current.isocalendar()[1] 
        reexp = r'y-.+W'
        extract = re.findall(reexp,expression)[0]

        # y-+1W
        if extract[2:-1] == '?':
            rep = str(W)
            history.update({'week':rep})
            return history
        elif extract[2] == '+':
            cur = W + int(extract[3:-1])
            if cur > 53:
                year_adjust = int(cur / 53)
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        elif extract[2] == '-':
            cur = W - int(extract[3:-1])
            if cur < 0:
                year_adjust = int(cur / 53) -1
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'week':rep})
            return history
    
    def create_from_D(self, history):
        if isinstance(history,dt.datetime):
            return history
        return dt.datetime(**history)
    
    def create_from_W(self, history):
        expression ='{}y-{}W-{}w-{}H:{}M:{}S'.format(history['year'],
                                                     history['week'],
                                                     history['weekday'],
                                                     history['hour'],
                                                     history['minute'],
                                                     history['second'])
        eval_time = dt.datetime.strptime(expression, "%Yy-%WW-%ww-%HH:%MM:%SS")
        return eval_time
    
    def ymd_reg(self,x):
        fix_ymd = r'(?:(?:今|明|后|大后)年)?(?:(?:\d{1,2}|下下下个|下下个|再下个|下个|十一|十二|一|二|三|四|五|六|七|八|九|十|后1个|后2个|后一个|后两个|后二个)月)?(?:(?:\d{1,2}|一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十|三十一)[日号])?'
        finds = list(set(re.findall(fix_ymd,x)) -set(['']))
        return finds

    def ymd_expression(self, result):
        def get_key(x):
            if x.isdigit():
                return x
            else:
                gets = time_dic.get(x)
                if x is None:
                    return '?'
                else:
                    return gets


        time_dic = {'今':'?','明':'+1','后':'+2','大后':'+3','下个':'+1','下下个':'+2','再下个':'+2','下下下个':'+3','后1个':'+1','后2个':'+1','后一个':'+1','后两个':'+1',
                    '一':'1','二':'2','三':'3','四':'4','五':'5','六':'6',
                   '七':'7','八':'8','九':'9','十':'10','十一':'11','十二':'12','十三':'13','十四':'14','十五':'15',
                   '十六':'16','十七':'17','十八':'18','十九':'19','二十':'20','二十一':'21','二十二':'22','二十三':'23',
                   '二十四':'24','二十五':'25','二十六':'26','二十七':'27','二十八':'28','二十九':'29','三十':'30','三十一':'31'}
        year_index = result.find('年')
        month_index = result.find('月')
        if result.find('日') != -1:
            date_index = result.find('日')
        else:
            date_index = result.find('号')

        if year_index != -1:
            year_key = result[0:year_index]  
            year = get_key(year_key)
        else:
            year = '?'
        if month_index != -1:
            month_key = result[year_index+1:month_index]
            month = get_key(month_key)
        else:
            month = '?'
        if date_index != -1:
            date_key = result[month_index+1:date_index]
            date = get_key(date_key)
        else:
            date = '?'
        formatted = '{}y-{}m-{}d-12H:00M:00S'.format(year, month, date)
        return formatted
    
    def evl_ymd(self,text):
        finds = self.ymd_reg(text)
        evls = []
        if len(finds) == 0:
            return evls
        else:
            for each in finds:
                evls.append({'pattern':each, 'expression':self.ymd_expression(each)})
        return evls
    
    def test1(self):
        month = ['下个月','再下个月','下下个月','1月','2月',
                 '3月','4月','5月','6月','7月','8月','9月','10月','11月','12月',
                 '一月','二月','三月','四月','五月','六月','七月','八月','九月','十月','十一月','十二月',]

In [110]:

class Test:
    def __init__(self):
        self.t = TimePattern(pattern_path='mapping.csv')
    
    def test_fixed_ymd(self):
        month = ['下个月','再下个月','下下个月','1月','2月',
                 '3月','4月','5月','6月','7月','8月','9月','10月','11月','12月',
                 '一月','二月','三月','四月','五月','六月','七月','八月','九月','十月','十一月','十二月']
        year = ['明年','今年','后年','大后年']
        day = ['1号','5号','7号','13号','18号','21号','24号','一号','二十三号',]
        for y in year:
            for m in month:
                for d in day:
                    string = y+m+d
                    try:
                        response = self.t.process(string)
                    except ValueError:
                        print(string)
                    assert len(response) == 1
t = Test()
t.test_fixed_ymd()

In [111]:
tt = TimePattern(pattern_path='mapping.csv', tz='America/New_York')

In [112]:
try:
    tt.process('12月32日')
except ValueError as e:
    if e.args[0] == 'day is out of range for month':
        print('您所指定的时间不在月份范围内')

您所指定的时间不在月份范围内


In [108]:
a.args

('day is out of range for month',)

In [92]:
t.process('我明年7月1号哦哦不下周一就还')

[{'gapH': 8874.747221632222,
  'gapS': 31949089.997876,
  'pattern': '明年7月1号',
  'time': datetime.datetime(2019, 7, 1, 16, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)}]

In [64]:
84608.408779 / 3600

23.502335771944445

In [18]:
%%timeit
a = t.evl('?y-+2W-3w-12H:00M:00S')


85.4 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
%%timeit
a = t.evl('?y-?m-?d-?H:00M:00S')

61.5 µs ± 895 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
df = pd.DataFrame([{'key_word':'明天','expression':'?y-?m-+1d-12H:00M:00S'}])

In [25]:
df.to_csv('mapping.csv',encoding='utf-8',index=False)

In [51]:
df = pd.read_csv('mapping.csv')
# create length
df['length'] = df.key_word.apply(lambda x: len(x))
df = df.sort_values(['length','key_word'], ascending=False)
df_series = pd.Series(index=df.key_word.values, data=df.expression.values)
df_dict = df_series.to_dict()

In [54]:
df_series.index.values

array(['明天下午', '后天下午', '两周后', '一周后', '2周后', '1周后', '明天', '后天'], dtype=object)

In [49]:
df = df.sort_values(['length','key_word'], ascending=False)

,expression,key_word,length
1,?y-?m-+1d-15H:00M:00S,明天下午,4
2,?y-?m-+2d-15H:00M:00S,后天下午,4
7,?y-+2W-?w-12H:00M:00S,两周后,3
4,?y-+1W-?w-12H:00M:00S,一周后,3
6,?y-+2W-?w-12H:00M:00S,2周后,3
5,?y-+1W-?w-12H:00M:00S,1周后,3
0,?y-?m-+1d-12H:00M:00S,明天,2
3,?y-?m-+2d-12H:00M:00S,后天,2
